# 示例：缓存及加载功能
- 可以缓存及加载任何对象（数据、模型……）

In [1]:
from fastphm.data.FeatureExtractor import FeatureExtractor
from fastphm.data.labeler.BearingRulLabeler import BearingRulLabeler
from fastphm.data.loader.bearing.XJTULoader import XJTULoader
from fastphm.data.processor.RMSProcessor import RMSProcessor
from fastphm.data.stage.BearingStageCalculator import BearingStageCalculator
from fastphm.data.stage.fpt.ThreeSigmaFPTCalculator import ThreeSigmaFPTCalculator
from fastphm.model.pytorch.base.BaseTrainer import BaseTrainer
from fastphm.model.pytorch.basic.MLP import MLP
from fastphm.system.Cache import Cache

### 生成数据

In [2]:
data_loader = XJTULoader('D:\\data\\dataset\\XJTU-SY_Bearing_Datasets')
feature_extractor = FeatureExtractor(RMSProcessor(data_loader.continuum))
fpt_calculator = ThreeSigmaFPTCalculator()
stage_calculator = BearingStageCalculator(data_loader.continuum, fpt_calculator)

# 获取原始数据、特征数据、阶段数据
bearing = data_loader("Bearing1_3", 'Horizontal Vibration')
feature_extractor(bearing)
stage_calculator(bearing)

generator = BearingRulLabeler(2048, time_ratio=60, is_from_fpt=False, is_rectified=True)
train_set = generator(bearing)

[DEBUG   13:52:59]  
[DataLoader]  Root directory: D:\data\dataset\XJTU-SY_Bearing_Datasets
	✓ Bearing1_1, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_1
	✓ Bearing1_2, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_2
	✓ Bearing1_3, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_3
	✓ Bearing1_4, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_4
	✓ Bearing1_5, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\35Hz12kN\Bearing1_5
	✓ Bearing2_1, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_1
	✓ Bearing2_2, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_2
	✓ Bearing2_3, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_3
	✓ Bearing2_4, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_4
	✓ Bearing2_5, location: D:\data\dataset\XJTU-SY_Bearing_Datasets\37.5Hz11kN\Bearing2_5
	✓ Bearing3_1, location: D:\data\datas

### 缓存数据

In [3]:
Cache.save(train_set, 'train_set')

[DEBUG   13:53:02]  [Cache]  Generating cache file: .\cache\train_set.pkl
[DEBUG   13:53:02]  [Cache]  Generated cache file: .\cache\train_set.pkl


### 加载数据

In [4]:
train_set = Cache.load('train_set')

[DEBUG   13:53:02]  [Cache]  -> Loading cache file: .\cache\train_set.pkl
[DEBUG   13:53:02]  [Cache]  ✓ Successfully loaded: .\cache\train_set.pkl


In [5]:
train_set.x

array([[ -0.77383518,   0.1305342 ,   0.1555443 , ...,   0.1136541 ,
         -0.54583549,   0.42616129],
       [ -0.38670301,  -0.3725886 ,  -0.74648857, ...,  -0.0345588 ,
         -0.70579052,  -0.79065561],
       [ -0.27912861,   0.52905079,  -0.2827883 , ...,  -0.63660137,
          0.25235411,   0.45748949],
       ...,
       [ -0.48875809,   0.66200487,  -2.59971619, ..., -11.16659641,
         -3.66904736,  -6.93216324],
       [ -2.04831362,  -0.154233  ,   3.88391018, ...,  -9.79976654,
          1.93562508,  -7.22073317],
       [ -3.97603512,   2.60374546,  -0.52655939, ...,  -5.99588156,
         -4.60753441, -10.23435593]])

### 生成模型并训练

In [6]:
trainer = BaseTrainer()
model = MLP(2048,16,1)
trainer.train(model,train_set)

[INFO    13:53:02]  
[Trainer]  Start training by BaseTrainer:
	training set: Bearing1_3
	callbacks: []
	device: cuda
	dtype: torch.float32
	epochs: 100
	batch_size: 256
	criterion: MSELoss()
	lr: 0.01
	weight_decay: 0.0
	optimizer: Adam
[INFO    13:53:03]  [BaseTrainer]  Epoch [1/100], MSELoss:2.0840
[INFO    13:53:03]  [BaseTrainer]  Epoch [2/100], MSELoss:1.5193
[INFO    13:53:03]  [BaseTrainer]  Epoch [3/100], MSELoss:0.7640
[INFO    13:53:03]  [BaseTrainer]  Epoch [4/100], MSELoss:0.5087
[INFO    13:53:03]  [BaseTrainer]  Epoch [5/100], MSELoss:0.3622
[INFO    13:53:03]  [BaseTrainer]  Epoch [6/100], MSELoss:0.2742
[INFO    13:53:03]  [BaseTrainer]  Epoch [7/100], MSELoss:0.2121
[INFO    13:53:03]  [BaseTrainer]  Epoch [8/100], MSELoss:0.1762
[INFO    13:53:03]  [BaseTrainer]  Epoch [9/100], MSELoss:0.1386
[INFO    13:53:03]  [BaseTrainer]  Epoch [10/100], MSELoss:0.1104
[INFO    13:53:03]  [BaseTrainer]  Epoch [11/100], MSELoss:0.0865
[INFO    13:53:03]  [BaseTrainer]  Epoch [12/

{'MSELoss': [2.084027421474457,
  1.5193231225013732,
  0.7640481412410736,
  0.508734080195427,
  0.36224835813045503,
  0.27415941953659057,
  0.21211764365434646,
  0.17620156556367875,
  0.1386438399553299,
  0.11043676733970642,
  0.0865045927464962,
  0.0659977313131094,
  0.05117107555270195,
  0.03954384885728359,
  0.031665900722146034,
  0.026047676056623458,
  0.022646864876151085,
  0.020868712477385996,
  0.01979600079357624,
  0.021487693581730128,
  0.026318890601396562,
  0.02589519917964935,
  0.024762030690908432,
  0.019007534254342318,
  0.018213863577693702,
  0.01960498243570328,
  0.01751903733238578,
  0.012640838138759136,
  0.00981745389290154,
  0.007396195316687226,
  0.005810825573280454,
  0.005492550041526556,
  0.005126125691458583,
  0.00455347269307822,
  0.004289551009424031,
  0.004221928515471518,
  0.003778877342119813,
  0.00362603475805372,
  0.0036148195154964923,
  0.0038201405899599195,
  0.003642614930868149,
  0.003639403195120394,
  0.00430

### 缓存模型

In [7]:
Cache.save(model, 'model')

[DEBUG   13:53:05]  [Cache]  Generating cache file: .\cache\model.pkl
[DEBUG   13:53:05]  [Cache]  Generated cache file: .\cache\model.pkl


### 加载模型

In [8]:
model = Cache.load('model')

[DEBUG   13:53:05]  [Cache]  -> Loading cache file: .\cache\model.pkl
[DEBUG   13:53:05]  [Cache]  ✓ Successfully loaded: .\cache\model.pkl
